## Table of Contents

---

## 구조적 API
구조적 API란?: 데이터의 흐름을 정의하는 기본 추상화 개념. 비정형 로그 파일부터 반정형 csv 파일, 정형적인 Parquet 파일까지 다양한 유형의 데이터를 처리 할 수 있는 도구.

#### 구조적 API의 종류
* Dataset
* DataFrame
* SQL Table & View

#### Schema
Schema: DataFrame의 column name과 data type을 정의. 
1. 데이터소스에서 얻거나
2. 직접 정의 할 수 있음

### Dataset vs DataFrame
* Dataset: '타입형' - 스키마의 명시된 데이터 타입의 일치 여부를 컴파일 타임에 확인
    * JVM기반의 언어인 스칼라와 자바에서만 지원
* DataFrame: '비타입형' - 스키마에 명시된 데이터 타입의 일치 여부를 런타임에 확인
    * Python을 포함한 대부분의 언어에서 지원
    
스파크는 자체 데이터 타입을 지원하는 여러 언어 API와 매핑된다.

따라서, DataFrame을 다른 언어로 작성하여 실행하여도 해당 데이터 타입이 스파크의 데이터 타입과 매핑되어, 스파크의 연산이 수행된다.

아래의 코드 예제도 파이썬이 아닌 **스파크**의 덧셈 연산으로 진행된다.

In [2]:
df= spark.range(500).toDF("number")
df.select(df["number"] + 10)

DataFrame[(number + 10): bigint]

---
## 구조적 API의 실행 과정
구체적으로 스파크 코드가 클러스터에서 어떤 과정을 거처 실행되는 지 알아보자. 크게는 다음과 같다.
0. DataFrame/Dataset/SQL을 이용한 코드 작성
1. (정상적인 코드라면) 스파크가 코드를 **논리적 실행 계획**으로 변환
2. 스파크는 **논리적 실행 계획**을 **물리적 실행 계획**으로 변환하며, 추가적인 최적화 가능성 탐색
3. 클러스터에서 물리적 실행 계획(RDD 처리)를 실행

### 1. 논리적 실행 계획
사용자의 코드를 최적화된 **논리적 실행 계획**으로 바꾸는 단계.

1. 사용자의 코드를 추상적 트랜스포메이션으로 변환. 코드의 유효성/테이블, 컬럼의 존재 여부만을 판단 - **검증 전 논리적 실행 계획**
2. 스파크 분석기: 필요한 테이블과 컬럼을 구체적으로 검증 - **검증된 논리적 실행 계획**
3. 카탈리스트 optimizer: 논리적 실행 계획을 최적화 - **최적화된 논리적 실행 계획**

### 2. 물리적 실행 계획
논리적 실행 계획을 클러스터 환경에서 실행하는 방법 정의.

다양한 물리적 실행 전략을 생성하고 비용 모델을 이용하여 비교한 후 최적의 전략 선택.

이러한 물리적 실행 계획은 일련의 RDD와 트랜스포메이션으로 변환.

---

In [1]:
from pyspark.sql import SparkSession
MAX_MEMORY = "32g"
spark = (
    SparkSession.builder.appName("Test Shell")
    .config("spark.driver.memory", MAX_MEMORY)
    .getOrCreate()
)